In [182]:
import requests
import json
import pandas as pd
import ta

In [181]:
#Daily Timeframe Data Grabber
def daily_df(ticker):

    API_URL = "https://www.alphavantage.co/query"

    data = {
        "function": "TIME_SERIES_DAILY",
        "symbol": ticker,
        "outputsize": "compact",
        "datatype": "json",
        "apikey": "PVGTC9P6IUS7XLQ9",
        }

    response = requests.get(API_URL, params=data)
    
    if response.status_code == 200:
        stock_data = response.json()
        meta_data = stock_data['Meta Data']
        daily_data = stock_data['Time Series (Daily)']
        daily_dataframe_rev = pd.DataFrame.from_dict(daily_data)
        daily_dataframe = daily_dataframe_rev.transpose()     
        daily_dataframe.columns = ['open', 'high', 'low', 'close', 'volume']
        df_entries2float(daily_dataframe)
        return daily_dataframe

In [180]:
#Converting dataframe entries to floats
def df_entries2float(df):
    for i in df.columns:
        column2convert = df[i]
        column_size = column2convert.size
        for k in range(0,column_size):
            column2convert[k] = float(column2convert[k])
    return df

In [190]:
#Reversing dataframe
def rev(df):
    return df.sort_index(ascending = True, axis = 0)

In [184]:
#Running TA with proper formatting
def tech_indicator(indicator, daily_df):
    indicator_df = pd.DataFrame(indicator)
    indicator_df = indicator_df.sort_index(ascending=False, axis=0)
    resulting_df = daily_df.join(indicator_df)
    return resulting_df

In [252]:
#All technical indicators in use
def run_indicators(df):
    high = rev(df['high'])
    low = rev(df['low'])    
    close = rev(df['close'])
    volume = rev(df['volume'])
    
    df = tech_indicator(ta.momentum.wr(high,low,close),df)
    df = tech_indicator(ta.momentum.money_flow_index(high,low,close,volume),df)
    df = tech_indicator(ta.momentum.stoch_signal(high,low,close),df)
    df = tech_indicator(ta.momentum.tsi(close),df)
    df = tech_indicator(ta.trend.macd(close),df)
    df = tech_indicator(ta.trend.trix(close),df)
    df = tech_indicator(ta.trend.aroon_up(close),df)
    df = tech_indicator(ta.trend.aroon_down(close),df)
    df = tech_indicator(ta.volume.force_index(close,volume),df) 
    df = tech_indicator(ta.volume.volume_price_trend(close,volume),df)
    df = tech_indicator(ta.volatility.keltner_channel_hband_indicator(high,low,close),df)
    df = tech_indicator(ta.volatility.keltner_channel_lband_indicator(high,low,close),df)
    df = tech_indicator((ta.volatility.bollinger_hband_indicator(close)),df)
    df = tech_indicator((ta.volatility.bollinger_lband_indicator(close)),df)    
    df['bbi'] = df['bbihband'].subtract(df['bbilband'])
    df['kci'] = df['kci_hband'].subtract(df['kci_lband'])
    del df['bbihband']
    del df['bbilband']
    del df['kci_hband']
    del df['kci_lband']
    
    return df

In [269]:
#Mapping values
def map_values(df):
    length = len(df)
    
    #Mapping wr values
    for i in range(length):
        if -100 <= df['wr'][i] <= -80:
            df['wr'][i] = 2
        elif -20<= df['wr'][i] <= 0:
            df['wr'][i] = -2
        elif -30 <= df['wr'][i] < -20:
            df['wr'][i] = -1
        elif -80 < df['wr'][i] <= -70:
            df['wr'][i] = 1
        else:
            df['wr'][i] = 0
            
    #Mapping MFI values
            
    return df

In [270]:
test3 = map_values(test2)
test3.head(10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\ProgramData\Anaconda3

,open,high,low,close,volume,wr,mfi_14,stoch_d,tsi,MACD_12_26,trix_15,aroon_up25,aroon_down25,fi_2,vpt,bbi,kci
2019-10-25,130.5,131.76,130.03,130.9,5.4432e+06,0,58.701819,48.102017,-11.099857,-0.633093,-0.114131,12.0,36.0,121348,-11380.8,0.0,0.0
2019-10-24,131.52,131.7,129.63,130.26,5.7463e+06,0,58.851098,57.641779,-11.327050,-0.693347,-0.124218,16.0,40.0,7.58694e+06,-95397.4,0.0,0.0
2019-10-23,132.45,132.68,130.76,131.13,5.9708e+06,0,65.616422,59.960375,-11.402530,-0.695030,-0.134766,4.0,44.0,-249081,95376,0.0,0.0
2019-10-22,133.15,133.68,132.1,132.4,9.2916e+06,-1,63.964750,60.678150,-11.676242,-0.775610,-0.147093,4.0,48.0,902376,122532,0.0,0.0
2019-10-21,131.44,131.73,130.02,130.26,6.2571e+06,0,54.556053,61.581921,-12.255095,-0.996424,-0.159768,8.0,52.0,3.68828e+06,-127322,0.0,0.0
2019-10-18,132.37,133.16,130.89,130.89,8.694e+06,0,54.733044,72.011115,-12.310172,-1.045994,-0.169815,12.0,56.0,32496,-4834.47,0.0,0.0
2019-10-17,132.53,133.44,130.51,132.37,8.0051e+06,-2,63.006645,65.931784,-12.504637,-1.158156,-0.180448,4.0,60.0,5.45594e+06,156889,0.0,0.0
2019-10-16,129.76,131.83,129.7,130.86,7.6108e+06,-1,53.664235,50.888546,-13.030084,-1.435068,-0.190399,8.0,64.0,4.52458e+06,67254.4,0.0,0.0
2019-10-15,130.01,130.89,129.4,129.76,5.9147e+06,0,43.075186,37.598424,-13.202092,-1.613992,-0.196019,12.0,68.0,167518,-6395.47,0.0,0.0
2019-10-14,130.13,130.3,129.48,129.7,3.7103e+06,0,43.523036,33.828357,-13.097532,-1.706966,-0.198677,16.0,72.0,-288612,25352,0.0,0.0


In [261]:
test = daily_df('DIS')
test2 = run_indicators(test)
test2.head(10)

,open,high,low,close,volume,wr,mfi_14,stoch_d,tsi,MACD_12_26,trix_15,aroon_up25,aroon_down25,fi_2,vpt,bbi,kci
2019-10-25,130.5,131.76,130.03,130.9,5.4432e+06,-49.4662,58.701819,48.102017,-11.099857,-0.633093,-0.114131,12.0,36.0,121348,-11380.8,0.0,0.0
2019-10-24,131.52,131.7,129.63,130.26,5.7463e+06,-60.8541,58.851098,57.641779,-11.327050,-0.693347,-0.124218,16.0,40.0,7.58694e+06,-95397.4,0.0,0.0
2019-10-23,132.45,132.68,130.76,131.13,5.9708e+06,-45.3737,65.616422,59.960375,-11.402530,-0.695030,-0.134766,4.0,44.0,-249081,95376,0.0,0.0
2019-10-22,133.15,133.68,132.1,132.4,9.2916e+06,-20.8469,63.964750,60.678150,-11.676242,-0.775610,-0.147093,4.0,48.0,902376,122532,0.0,0.0
2019-10-21,131.44,131.73,130.02,130.26,6.2571e+06,-53.8983,54.556053,61.581921,-12.255095,-0.996424,-0.159768,8.0,52.0,3.68828e+06,-127322,0.0,0.0
2019-10-18,132.37,133.16,130.89,130.89,8.694e+06,-43.2203,54.733044,72.011115,-12.310172,-1.045994,-0.169815,12.0,56.0,32496,-4834.47,0.0,0.0
2019-10-17,132.53,133.44,130.51,132.37,8.0051e+06,-18.1356,63.006645,65.931784,-12.504637,-1.158156,-0.180448,4.0,60.0,5.45594e+06,156889,0.0,0.0
2019-10-16,129.76,131.83,129.7,130.86,7.6108e+06,-22.6107,53.664235,50.888546,-13.030084,-1.435068,-0.190399,8.0,64.0,4.52458e+06,67254.4,0.0,0.0
2019-10-15,130.01,130.89,129.4,129.76,5.9147e+06,-61.4583,43.075186,37.598424,-13.202092,-1.613992,-0.196019,12.0,68.0,167518,-6395.47,0.0,0.0
2019-10-14,130.13,130.3,129.48,129.7,3.7103e+06,-63.2653,43.523036,33.828357,-13.097532,-1.706966,-0.198677,16.0,72.0,-288612,25352,0.0,0.0
